In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://ucdavisaggies.com/sports/mens-basketball/stats'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [3]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [4]:
#ISSUE: COLUMNS ARE OUT OF ORDER
#After #, GP, GS it goes out of order
    #It jumps to 

#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)


[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [5]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data




['02', 'Johnson, TY', '32', '32', '1077', '33.7', '229', '591', '.387', '39', '139', '.281', '178', '232', '.767', '675', '21.1', '28', '104', '132', '4.1', '75', '96', '124', '72', '7', 'View Bio']
['08', 'Tamba, Pablo', '32', '32', '883', '27.6', '138', '235', '.587', '4', '16', '.250', '81', '123', '.659', '361', '11.3', '49', '119', '168', '5.3', '84', '35', '73', '37', '24', 'View Bio']
['04', 'Sevilla, Connor', '32', '24', '936', '29.3', '90', '241', '.373', '63', '174', '.362', '41', '45', '.911', '284', '8.9', '16', '35', '51', '1.6', '50', '29', '26', '16', '1', 'View Bio']
['10', 'DeBruhl, Leo', '32', '32', '918', '28.7', '87', '239', '.364', '44', '114', '.386', '62', '73', '.849', '280', '8.8', '27', '89', '116', '3.6', '83', '94', '75', '44', '4', 'View Bio']
['07', 'Rocak, Niko', '32', '27', '801', '25.0', '56', '109', '.514', '11', '32', '.344', '49', '69', '.710', '172', '5.4', '64', '152', '216', '6.8', '94', '43', '44', '27', '58', 'View Bio']
['01', '01 Daughtery Jr.

In [7]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/csuf_player_stats.csv', index=False, encoding = "utf-8")